In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import plotly.plotly as py
import plotly.graph_objs as go
from plotly import tools
from scipy import stats
import matplotlib.pyplot as plt
sns.set(style="whitegrid")
%matplotlib inline
import datetime, warnings, scipy 
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib.patches import ConnectionPatch
import nltk
from sklearn import preprocessing
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelBinarizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from wordcloud import WordCloud,STOPWORDS
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize,sent_tokenize
from bs4 import BeautifulSoup
import spacy
from sklearn.model_selection import train_test_split
import re,string,unicodedata
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.stem import LancasterStemmer,WordNetLemmatizer
from sklearn.linear_model import LogisticRegression,SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
from collections import OrderedDict
from matplotlib.gridspec import GridSpec
#from mpl_toolkits.basemap import Basemap
from scipy.optimize import curve_fit
plt.rcParams["patch.force_edgecolor"] = True
plt.style.use('fivethirtyeight')
mpl.rc('patch', edgecolor = 'dimgray', linewidth=1)
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "last_expr"
pd.options.display.max_columns = 50
warnings.filterwarnings("ignore")
from glob import glob

In [ ]:
import nltk
import spacy
import string
pd.options.mode.chained_assignment = None

In [ ]:
neg_text = sorted(glob("C:/Users/lohitr/Desktop/Bits_Data_Visualze/review_polarity/txt_sentoken/neg/*.txt"))

In [ ]:
neg_text

In [ ]:
text = pd.read_csv("C:/Users/lohitr/Desktop/Bits_Data_Visualze/IMDB Dataset.csv")

In [ ]:
text.columns

In [ ]:
text.info()

In [ ]:
text.dtypes

In [ ]:
text.describe()

In [ ]:
text.head()

In [ ]:
text.tail()

In [ ]:
X=text.review 
y=text.sentiment

In [ ]:
# create training and testing vars
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

In [ ]:
corpus = []
def text_data():
    text = pd.read_csv("C:/Users/lohitr/Desktop/Bits_Data_Visualze/IMDB Dataset.csv")
    X=text.review 
    for i in X:
        review = re.sub('[^a-zA-Z0-9\s]', ' ', i)
        review = review.lower()
        review = review.split()
        stop_words = set(stopwords.words("english"))  # load stopwords
        review = filter(lambda x: x not in stop_words, review)
        review = ' '.join(review)
        corpus.append(review)
        #print(corpus)
    #return corpus
        
abc=text_data()

In [ ]:
print(corpus)



In [ ]:
X_train=corpus[:40000]
y=corpus[40000:]


In [ ]:
#Count vectorizer for bag of words

VectorCount=CountVectorizer(min_df=0,max_df=1,binary=False,ngram_range=(1,3))

VectorCount_train_reviews=VectorCount.fit_transform(X_train)

VectorCount_test_reviews=VectorCount.transform(y)

print('Bag_Words_VectorCount_train:',VectorCount_train_reviews.shape)
print('Bag_Words_VectorCount_test:',VectorCount_test_reviews.shape)

In [ ]:
#Tfidf vectorizer
tv=TfidfVectorizer(min_df=0,max_df=1,use_idf=True,ngram_range=(1,3))
#transformed train reviews
tv_train_reviews=tv.fit_transform(X_train)
#transformed test reviews
tv_test_reviews=tv.transform(y)
print('Tfidf_train:',tv_train_reviews.shape)
print('Tfidf_test:',tv_test_reviews.shape)

In [ ]:
label_encoder = preprocessing.LabelEncoder()
text['sentiment'] = label_encoder.fit_transform(text['sentiment'])
#transformed sentiment data
sentiment_data=lb.fit_transform(text['sentiment'])
print(sentiment_data.shape)

train_sentiments=text[:40000]
test_sentiments=text[40000:]
print(train_sentiments)
print(test_sentiments)

In [ ]:
#labeling the sentient data
lb=LabelBinarizer()
#transformed sentiment data
sentiment_data=lb.fit_transform(text['sentiment'])
print(sentiment_data.shape)

In [ ]:
train_sentiments=sentiment_data[:40000]
test_sentiments=sentiment_data[40000:]
print(train_sentiments)
print(test_sentiments)

In [ ]:
################## Logistic Regression ####################
#training the model
lr=LogisticRegression(penalty='l2',max_iter=500,C=1,random_state=42)
#Fitting the model for Bag of words
lr_bow=lr.fit(VectorCount_train_reviews,train_sentiments)
print(lr_bow)
#Fitting the model for tfidf features
lr_tfidf=lr.fit(tv_train_reviews,train_sentiments)
print(lr_tfidf)

In [ ]:
#Predicting the model for bag of words
lr_bow_predict=lr.predict(VectorCount_test_reviews)
print(lr_bow_predict)
##Predicting the model for tfidf features
lr_tfidf_predict=lr.predict(tv_test_reviews)
print(lr_tfidf_predict)

In [ ]:
lr_bow_score=accuracy_score(test_sentiments,lr_bow_predict)
print("lr_bow_score :",lr_bow_score)
#Accuracy score for tfidf features
lr_tfidf_score=accuracy_score(test_sentiments,lr_tfidf_predict)
print("lr_tfidf_score :",lr_tfidf_score)

In [ ]:
#Classification report for bag of words 
lr_bow_report=classification_report(test_sentiments,lr_bow_predict,target_names=['Positive','Negative'])
print(lr_bow_report)

#Classification report for tfidf features
lr_tfidf_report=classification_report(test_sentiments,lr_tfidf_predict,target_names=['Positive','Negative'])
print(lr_tfidf_report)

In [ ]:
cm_bow=confusion_matrix(test_sentiments,lr_bow_predict,labels=[1,0])
print(cm_bow)
#confusion matrix for tfidf features
cm_tfidf=confusion_matrix(test_sentiments,lr_tfidf_predict,labels=[1,0])
print(cm_tfidf)

In [ ]:
######################### Multinomial ##########
multi=MultinomialNB()
#fitting the svm for bag of words
multi_bow=multi.fit(VectorCount_train_reviews,train_sentiments)
print(multi_bow)


#fitting the svm for tfidf features
multi_tfidf=multi.fit(tv_train_reviews,train_sentiments)
print(multi_tfidf)


#Predicting the model for bag of words
multi_bow_predict=multi.predict(VectorCount_test_reviews)
print(multi_bow_predict)
#Predicting the model for tfidf features
multi_tfidf_predict=multi.predict(tv_test_reviews)
print(multi_tfidf_predict)